In [ ]:
import pandas as pd
from tqdm import tqdm
train_path = "../Data/NER_data/BIO_NETrain.txt"
test_path = "../Data/NER_data/BIO_NETest.txt"



df_train = pd.read_csv(train_path, sep= "\t", header=None)
df_train.columns = ["words", "labels"]
df_train.dropna(inplace=True)

df_test = pd.read_csv(test_path, sep= "\t", header=None)
df_test.columns = ["words", "labels"]
df_test.dropna(inplace=True)

df = 

In [ ]:
df["sentence_idx"] = -1

last_index = 0
mini_df = df[df.words == "EOS"]
for i in tqdm(range(mini_df.shape[0])):
    df.loc[last_index : mini_df.index.values[i], "sentence_idx"] = i
    last_index =  mini_df.index.values[i] + 1

In [ ]:
df.tail(20)

In [ ]:
class SentenceGetter(object):
    
    def __init__(self, dataset):
        self.n_sent = 1
        self.dataset = dataset
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s["words"].values.tolist(),
                                                       #s['pos'].values.tolist(),
                                                        s["labels"].values.tolist())]
        self.grouped = self.dataset.groupby("sentence_idx").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [ ]:
getter = SentenceGetter(df)
sentences = getter.sentences

In [ ]:
sentences[0][i][0]

In [ ]:
txt = ""
for i in range(len(sentences[0]) - 1):
    txt = txt + " " + sentences[0][i][0]

In [ ]:
txt

In [ ]:
TurkishMorphology morphology = TurkishMorphology.createWithDefaults();
String sentence = "Yarın kar yağacak.";
System.out.println("Sentence  = " + sentence);
List<WordAnalysis> analysis = morphology.analyzeSentence(sentence);

System.out.println("Before disambiguation.");
for (WordAnalysis entry : analysis) {
  System.out.println("Word = " + entry.getInput());
  for (SingleAnalysis single : entry) {
    System.out.println(single.formatLong());
  }
}

In [ ]:
maxlen = max([len(s) for s in sentences])
print ('Maximum sequence length:', maxlen)

In [ ]:
# Check how long sentences are so that we can pad them
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("ggplot")
plt.hist([len(s) for s in sentences], bins=100)
plt.show()

In [ ]:
words = list(set(df["words"].values))
words.append("ENDPAD")
n_words = len(words);
n_words

In [ ]:
tags = list(set(df["labels"].values))
n_tags = len(tags)
n_tags

In [ ]:
tags

In [ ]:
word2idx = {w: i for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate(tags)}

In [ ]:
from keras.preprocessing.sequence import pad_sequences
X = [[word2idx[w[0]] for w in s] for s in sentences]

In [ ]:
X = pad_sequences(maxlen=140, sequences=X, padding="post",value=n_words - 1)
y = [[tag2idx[w[1]] for w in s] for s in sentences]
y = pad_sequences(maxlen=140, sequences=y, padding="post", value=tag2idx["O"])

In [ ]:
from keras.utils import to_categorical
y = [to_categorical(i, num_classes=n_tags) for i in y]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
import numpy as np
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional

input = Input(shape=(140,))
model = Embedding(input_dim=n_words, output_dim=140, input_length=140)(input)
model = Dropout(0.1)(model)
model = Bidirectional(LSTM(units=100, return_sequences=True, recurrent_dropout=0.1))(model)
out = TimeDistributed(Dense(n_tags, activation="softmax"))(model)  # softmax output layer
model = Model(input, out)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
history = model.fit(X_train, np.array(y_train), batch_size=32, epochs=1, validation_split=0.2, verbose=1)

In [ ]:
i = 0
p = model.predict(np.array([X_test[i]]))
p = np.argmax(p, axis=-1)
print("{:15} ({:5}): {}".format("Word", "True", "Pred"))
for w,pred in zip(X_test[i],p[0]):
    print("{:15}: {}".format(words[w],tags[pred]))

In [ ]:
p[0]

In [1]:
import jpype

def start_jvm_load_zemberek():
    jpype.startJVM("C:/Program Files/Java/jre1.8.0_191/bin/server/jvm.dll",
         "-Djava.class.path=../Other/zemberek_0_15.jar", "-ea")

    Tr = jpype.JClass("zemberek.morphology.TurkishMorphology")
    morphology = Tr.createWithDefaults()
    return morphology

        
def jvm_shutdown():
    jpype.shutdownJVM()
    

In [3]:
morphology = start_jvm_load_zemberek()
results = morphology.analyzeAndDisambiguate("nehirden atlarla atladık").bestAnalysis()

In [26]:
word = results[0].surfaceForm()
pos = str(results[0].dictionaryItem.primaryPos)
print(word, pos)

nehirden Noun


In [27]:
for i in range(len(results)):
    word = results[i].surfaceForm()
    pos = str(results[i].dictionaryItem.primaryPos)
    print(word, pos)

nehirden Noun
atlarla Noun
atladık Verb
